In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
import re
import sys
import traceback

sns.set()


def gethighest(vals, prefix="logall_"):
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
        return vals[m]

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

        
def readparams(file):
    params = {}
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

In [37]:
from IPython.display import HTML, display
from ipywidgets import widgets
a = widgets.Text()
import uuid
classname='user-agent-content'+str(uuid.uuid4())
a.add_class(classname)
display(a)

Text(value='', _dom_classes=('user-agent-content33802afe-142c-44e8-811f-bdb4348efc43',))

In [38]:
HTML("""
<script>
'use strict';
// Get the element
var inp = document.getElementsByClassName(
     '""" + str(classname) + """')[0].getElementsByTagName('input')[0];

// Set the UA
inp.value = window.navigator.userAgent;

// Trigger change for Jupyter lab kernel
var evt = new Event('change', {"bubbles":true, "cancelable":false});
inp.dispatchEvent(evt);
</script>
""")

In [39]:
if 'Mac OS' in a.value :
    %config InlineBackend.figure_format = 'retina'
    print('retina set')
else:
    print('nothing changed')

retina set


In [41]:
import tqdm.notebook
try:
    params
except NameError:
    params = []
else:
    print('params already exist, you need to force clear it')
    clearparam = input('Clear params? Y/N')
    if clearparam.upper() == 'Y':
        params = []
        print('params have been cleared')
    else:
        print('params have not been cleared')
prefix = '/home/pecoffet/remoterobo'
prefix = '/home/ecoffet/robocoop'
gen = 1499
runname = 'lionscross24-mean5paperready'
runalias = input(f'run name? ({runname})')
if not runalias:
    runalias = runname

params already exist, you need to force clear it


Clear params? Y/N Y


params have been cleared


run name? (lionscross24-mean5paperready) 


In [42]:
import time
from collections import defaultdict
paths = sorted([]
                  + glob.glob(f"{prefix}/logs/{runname}/**/rep00/", recursive=True)
                  )
#paths = sorted(glob.glob(f"{prefix}/logs/lions-nvar-*-2019-06-*/**/rep00/", recursive=True))
timers = defaultdict(lambda: 0)
start = time.monotonic()
current = start
alreadythere = set(param['path'] for param in params)
merge = True
if merge:
    try:
        alreadycomputedf = pd.read_pickle(f'all_file_loaded_{runalias.replace("*", "")}.pkl.gz')

        try:
            print('merging')
            alreadythere.update(set(alreadycomputedf['path']))
        except KeyError:
            pass
    except FileNotFoundError:
        alreadycomputedf = pd.DataFrame()
else:
    print('Merge is disabled, it can slow down the process')
    alreadycomputedf = pd.DataFrame()
        
for path in tqdm.notebook.tqdm(list(path for path in paths if path not in alreadythere)):
    

    #############
    # Read conf #
    #############
    curparam = readparams(glob.glob(path + 'properties*')[0])
    
    ###############
    # Get Fitness #
    ###############
    fitness = read_csvx(path + "/../fit.txt", delimiter="\t", names=['gen', 'min', 'q1', 'med', 'q3', 'max'])
    timers['readFit'] += time.monotonic() - current
    current = time.monotonic()
    
    
    #################
    # output graphs #
    #################
    if (False and curparam['maxPlayer'] >= 80 and curparam['gNbOfPhysicalObjects'] >= 40):
        print('coucou', path)
        plt.figure()
        fitness.plot(x='gen', y='med')
        plt.show()
        

    try:
        logall = read_csvx(path+f"/logall_{gen}.txt", delimiter="\t")
    except FileNotFoundError:
        #print('not finished yet for', path)
        continue
    timers['readLogall'] += time.monotonic() - current
    current = time.monotonic()
    

    ################
    # Process data #
    ################
    medfit = fitness.query(f"gen == {gen}")['med'].median()
    #meanfit = fitness.query(f"gen == {gen}")['fitness'].mean()

    
    coopoptis = logall.query(f'nbOnOpp == {curparam["nOpti"]}').groupby('id', as_index=False)['curCoopNoCoef'].apply(lambda x: x.iloc[0])
    nmod = logall['nbOnOpp'].mode()[0]
    coopabove2 = logall.query('nbOnOpp >= 2')['curCoopNoCoef'].mean()
    if coopoptis.empty or np.isnan(coopoptis.mean()):  # WARNING, bold choice !
        coopopti = 0
    else:
        coopopti = coopoptis.mean()
    coopmod =  logall.query(f'nbOnOpp == {nmod}').groupby('id', as_index=False)['curCoopNoCoef'].mean()['curCoopNoCoef'].mean()
    params.append({'path':path, 'params':curparam, 'coopopti': coopopti, 'coopmod': coopmod, 'coopabove2': coopabove2,
                   'nmod': nmod, 'medfit': medfit, 'meanfit': medfit})
    timers['processData'] += time.monotonic() - current
    current = time.monotonic()
else:
    print('everything already there')
end = time.monotonic()

print(timers)
print(end - start)

processed = []
df = pd.concat((pd.DataFrame(processed), alreadycomputedf))
excluded = []
for elem in params:
    param = elem['params']
    ess = param['meanA'] / param['nOpti']
    so = param['meanA'] + param['b'] * (param['nOpti'] - 1) / param['nOpti']
    outdict = {'coopopti': (elem['coopopti'] - ess) / (so - ess) , 'truecoopopti': elem['coopopti'],
               'coopmod': (elem['coopmod'] - ess) / (so - ess) , 'truecoopmod': elem['coopmod'], 'nmod': elem['nmod'], 'medfit': elem['medfit'],
               'meanfit': elem['meanfit'], 'coopabove2': elem['coopabove2'], 'path': elem['path']}
    outdict.update(param)
    processed.append(outdict)
df = pd.concat((pd.DataFrame(processed), alreadycomputedf))
assert(df['path'].nunique() == len(df))
df.to_pickle(f'all_file_loaded_{runalias}.pkl.gz')

merging



everything already there
defaultdict(<function <lambda> at 0x7fb59de0eb00>, {'readFit': 680.5878541469574, 'readLogall': 1889.8340838998556, 'processData': 912.4745917916298})
3482.9792637079954


In [ ]:
if False:
    diffkeys = set()
    missingkeys = set()
    i=0
    for key in params[i]['params'].keys():
        try:
            if any(params[j]['params'][key] != params[i]['params'][key] for j in range(0, len(params) - 1)) and key not in diffkeys:
                diffkeys.add(key)
                print(key, 'added at step', i)
        except KeyError:
            missingkeys.add(key)

In [8]:
excluded = []
processed = []
for elem in params:
    param = elem['params']
    ess = param['meanA'] / param['nOpti']
    so = param['meanA'] + param['b'] * (param['nOpti'] - 1) / param['nOpti']
    outdict = {'coopopti': (elem['coopopti'] - ess) / (so - ess) , 'truecoopopti': elem['coopopti'],
               'coopmod': (elem['coopmod'] - ess) / (so - ess) , 'truecoopmod': elem['coopmod'], 'nmod': elem['nmod'], 'medfit': elem['medfit'],
               'meanfit': elem['meanfit'], 'coopabove2': elem['coopabove2'], 'path': elem['path']}
    outdict.update(param)
    processed.append(outdict)
df = pd.concat((pd.DataFrame(processed), alreadycomputedf))
assert(df['path'].nunique() == len(df))
df.to_pickle(f'all_file_loaded_{runalias}.pkl.gz')

In [12]:
print(f'all_file_loaded_{runalias.replace("*", "")}.pkl.gz')
df = pd.read_pickle(f'all_file_loaded_{runalias.replace("*", "")}.pkl.gz')
#normalized_df=(df-df.mean())/df.std()

all_file_loaded_lionscross26-costtime.pkl.gz


In [ ]:
%matplotlib inline
sns.set_context('paper')
sns.set_style('white')
from colour import Color

## Colors
coopColor = Color("#3375b2")
defectColor = Color("#963d35")
badtoomany = Color("#963d35")
badtoomany.luminance *= 2
badnotenough = Color("#963d35")
badnotenough.luminance *= 1
good = Color("#3375b2")


## Helpers
def plot_ess(data, *args, **kwargs):
    try:
        nopti = data['nOpti'].mean()
    except KeyError:
        nopti = 2
    plt.axhline(5/nopti, *args, **kwargs)
    
def plot_so(data, *args, **kwargs):
    try:
        nopti = data['nOpti'].mean()
    except KeyError:
        nopti = 2
    plt.axhline(5 + 10 * (nopti-1)/nopti, *args, **kwargs)

def plot_span(data, *args, **kwargs):
    try:
        nhat = data['nOpti'].mean()
    except KeyError:
        nhat = 2
    omega = data['gNbOfPhysicalObjects'].mean()
    vs = plt.axvspan(0, nhat+1, alpha=0.1, color=badnotenough.hex, label="Not enough agents")
    vs.set_zorder(0)
    vs = plt.axvspan(nhat+1, min(nhat*omega+1, 102), alpha=0.1, color=good.hex, label="Enough opportunities and agents")
    vs.set_zorder(0)
    vs = plt.axvspan(min(nhat*omega+1, 102), 102, alpha=0.1, color=badtoomany.hex, label="Not enough opportunities")
    vs.set_zorder(0)

    
def annotator(itera, *args, size=15, **kwargs):
    plt.text(-0.05, 1.05, next(itera) + '.', transform=plt.gca().transAxes, 
            size=size, weight='bold')


In [ ]:
import string
dfcond = df
g = sns.relplot(data=dfcond, row="nOpti", col="gNbOfPhysicalObjects", x="maxPlayer", y='truecoopopti' , kind="line",
                facet_kws={'ylim':(-0.5, 12.5), 'xlim':(0, 102), 'sharey': True, 'legend_out': True}, aspect=1.2, height=2.5)
g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopopti")
#g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopmod", color="green")

g.map_dataframe(plot_ess,  color=defectColor.hex, label="defect investment")
g.map_dataframe(plot_so,  color=coopColor.hex, label="social optimum investment")
g.map_dataframe(plot_span)
g.map(annotator, itera=iter(string.ascii_lowercase))
g.set(ylabel='Mean investment for $\hat{n}$', xlabel="Number of agents in the environment")
g.set_titles("$\omega = {col_name} | \hat{{n}} = {row_name}$")

#g.set_xlabel('Number of agents in the environment')
g.add_legend()
g.savefig('out/varNrowOpp.pdf')
plt.show()

In [ ]:
nopti = 4
ntol = 0.7
condrec = f'cost == 5 and gNbOfPhysicalObjects == 80'
curcond = df.query(condrec)
vi='maxPlayer'
vilab='Nombre d\'agents dans l\'environnement'
fig, gax = plt.subplots(2, 1, figsize=(6, 3*2))
axs = gax
st = fig.suptitle("Avec 20 opportunités, après 1500 générations")
curplot = 0
sns.scatterplot(x=vi, y='coopopti', data=curcond, ax=axs[curplot], alpha=0.7)
sns.lineplot(x=vi, y='coopopti', data=curcond, ax=axs[curplot])
axs[curplot].set_ylabel('Coop moyenne des agents')
axs[curplot].set_xlabel(vilab)
axs[curplot].axhline(0, label='defect', c='r')
axs[curplot].axhline(1, label='SO', c='b')
axs[curplot].legend()

curplot += 1

if False:
    sns.regplot(x=vi, y='nmod', data=curcond, ax=axs[curplot])
    axs[curplot].set_ylabel('Mode du nombre d\'agent par opp')
    axs[curplot].set_xlabel(vilab)
    #axs[curplot].set_ylim(0, 10)
    axs[curplot].axhline(nopti, label='nopti', c='b')
    axs[curplot].legend()
    curplot += 1

sns.scatterplot(x=vi, y='medfit', data=curcond, ax=axs[curplot], alpha=0.7)
sns.lineplot(x=vi, y='meanfit', data=curcond, ax=axs[curplot])
axs[curplot].set_ylabel('Fitness moyenne des agents')
axs[curplot].set_xlabel(vilab)
fig.tight_layout()
st.set_y(0.95)
fig.subplots_adjust(top=0.90)
plt.show(fig);


In [ ]:
import statsmodels.formula.api as sm
sns.regplot(x='nOpti', y='truecoopopti', data=df.query(f'nTolerance == {ntol} and gNbOfPhysicalObjects == 20.0 and maxPlayer == 30'))
sm.ols(formula='truecoopopti ~ nOpti', data=df.query(f'nTolerance == {ntol} and gNbOfPhysicalObjects == 20.0 and maxPlayer == 30')).fit().summary()

In [ ]:
sns.regplot(x='nOpti', y='medfit', data=df.query(f'maxPlayer == 30'))
sm.ols(formula='medfit ~ nOpti', data=df.query(f'maxPlayer == 30')).fit().summary()

In [ ]:
#normalized_df['nbRobots'] = normalized_df['gInitialNumberOfRobots']
normalized_df['nbObj'] = normalized_df['gNbOfPhysicalObjects']

In [ ]:
result = sm.ols(formula="truecoopopti ~ maxPlayer + nOpti + nTolerance", data=normalized_df).fit()
result.summary()

In [ ]:
result = sm.ols(formula="medfit ~ gNbOfPhysicalObjects + nOpti + oppDecay", data=normalized_df).fit()
result.summary()

In [ ]:

def bellcurve(x, mu, sigma):
    return  1.0 / np.sqrt(2 * np.pi) * 1.0 / sigma * np.exp(- ((x - mu) * (x - mu)) / (2 * sigma * sigma))

def alonepayoff(x):
    a = 5
    b = 10
    return a * x - 0.5 * x**2

def alonepayoffbellcurve(mu, sigma):
    a = 5
    b = 10
    x = np.linspace(0, 10, 1000)
    xs, mus = np.meshgrid(x, mu)
    xs, sigmas = np.meshgrid(x, sigma)
    truemax = np.max((a * xs) * bellcurve(1, mus, sigmas) / bellcurve(1, 1, sigmas)  - 0.5 * xs**2, axis=1)
    return truemax.flatten()

In [ ]:
alonepayoffbellcurve(2, 2)

In [ ]:
ntol=0.7

df['ratio'] = (df['maxPlayer'])
df['betteralone'] = alonepayoffbellcurve(df['nOpti'], df['nTolerance'])
df['difffit'] = (df['meanfit'] - df['betteralone'])

fig, axs = plt.subplots(3, 1, figsize=(5, 12))
st = fig.suptitle(f"Tolerance={ntol}", fontsize=14)

condrec = f'nTolerance == {ntol} and gNbOfPhysicalObjects == 20'

curcond = df.query(condrec)
sns.heatmap(curcond.pivot_table('meanfit', 'nOpti', 'maxPlayer'), cmap=sns.cm.rocket_r, ax=axs[0])
axs[0].set_ylabel('Nombre optimal par opp')
axs[0].set_xlabel('Nombre d\'opportunités')
axs[0].set_title('Fitness Moyenne')


sns.heatmap(curcond.pivot_table('truecoopopti', 'nOpti', 'maxPlayer'), cmap=sns.cm.rocket_r, vmin=0, vmax=10, ax=axs[1])
axs[1].set_ylabel('Nombre optimal par opp')
axs[1].set_xlabel('Nombre d\'opportunités')
axs[1].set_title('Coopération Moyenne')

sns.heatmap(curcond.pivot_table('difffit', 'nOpti', 'maxPlayer'), linewidth=0.01, cmap=sns.cm.rocket_r, ax=axs[2])
axs[2].set_ylabel('Nombre optimal par opp')
axs[2].set_xlabel('Nombre d\'opportunités')
axs[2].set_title('Différence entre fitness seule espérée et fitness moyenne')
fig.tight_layout()

# shift subplots down:
st.set_y(0.95)
fig.subplots_adjust(top=0.90)
plt.show(fig)

In [ ]:
df.query('nOpti == 2 and gNbOfPhysicalObjects == 10')

In [ ]:
df.query('nOpti == 2 and gNbOfPhysicalObjects == 10')

In [ ]:
sns.regplot(y='truecoopopti', x='oppDecay', data=df.query('nOpti == 2 and nTolerance == 0.2 and gNbOfPhysicalObjects == 60'))
plt.ylabel('Coopération moyenne en fin de gen')
plt.xlabel('Durée de vie moyenne des opportunités')

In [ ]:
alonepayoffbellcurve(df['nOpti'], df['nTolerance'])

In [ ]:
df.iloc[-1]

In [ ]:
alonepayoffbellcurve(4, 2)

In [ ]:
toread = np.random.choice(list(alreadythere), 50, replace=False)

In [ ]:
%%timeit

for path in toread:
    read_csvx(path + '/logall_1499.txt', delimiter='\t')

In [ ]:
for i, path in enumerate(toread):
    a = read_csvx(path + '/logall_1499.txt', delimiter='\t')
    a.to_hdf('test/log' + str(i) + '.hdf5.gz', 'log')

In [ ]:
%%timeit

for i, path in enumerate(toread):
    a = pd.read_hdf(f'test/log{i}.hdf5.gz')

In [ ]:
a

In [ ]:
(2.86 * 1000) / 488 